In [1]:
from Bimatrix import PayoffMatrix
import torch

In [2]:
PayoffMatrix(10,5)

TypeError: zeros() received an invalid combination of arguments - got (tuple, dtype=ABCMeta), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
